

# Hyperparameter Tuning Practice

## *Data Science Unit 4 Sprint 2 Assignment 3*

# Gridsearch Hyperparameters

In the guided project, you learned how to use sklearn's `GridsearchCV` and `keras-tuner` libraries to tune the hyperparameters of a neural network model. For your module project you'll continue using these two libraries, however we are going to make things a little more interesting for you. 

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. 



**Don't forget to switch to GPU on Colab!**

## 0.1 Imports and installs

In [1]:
# native python libraries imports 
import math
from time import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# sklearn imports 
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# keras imports 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu, sigmoid
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import get_file

# required for compatibility between sklearn and keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# install keras-tuner
from kerastuner.tuners import RandomSearch, BayesianOptimization
from kerastuner.engine.hyperparameters import HyperParameters

/var/folders/l5/9b73nm7x0d5bz481kpj_k3640000gn/T/ipykernel_7347/2835263299.py:24: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch, BayesianOptimization


## 0.2 Load quickdraw data

In [2]:
def load_quickdraw10():
    """
    Fill out this doc string, and comment the code, for practice in writing the kind of code that will get you hired. 
    """
    
#     URL_ = "https://github.com/LambdaSchool/DS-Unit-4-Sprint-2-Neural-Networks/blob/main/quickdraw10.npz?raw=true"
    
#     path_to_zip = get_file('./quickdraw10.npz', origin=URL_, extract=False)

    data = np.load('../quickdraw10.npz')
    
    # normalize your image data
    max_pixel_value = 255
    X = data['arr_0']/max_pixel_value
    Y = data['arr_1']
        
    return train_test_split(X, Y, shuffle=True)

In [3]:
X_train, X_test, y_train, y_test = load_quickdraw10()

In [4]:
X_train.shape

(75000, 784)

In [5]:
y_train.shape

(75000,)

_____

# Experiment 1

## Tune Hyperperameters using Enhanced GridsearchCV 

We are going to use GridsearchCV again to tune a deep learning model however we are going to add some additional functionality to our gridsearch. 

Specifically, we are going to automate the generation of how many nodes to use in a layer and how many layers to use in a model! 

By the way, yes, there is a function within a function. Try to not let that bother you. An alternative to this would be to create a class. If you're up for the challenge give it a shot. However, consider this a stretch goal that you come back to after you finish going through this assignment. 


### Objective 

The objective of this experiment is to show you how to automate the generation of layers and layer nodes for the purposes of gridsearch. <br>
Up until now, we've been manually selecting the number of layers and layer nodes.

In [6]:
# Function to create model, required for KerasClassifier
def create_model(n_layers,  first_layer_nodes, last_layer_nodes, act_funct ="relu", negative_node_incrementation=True):
    """"
    Returns a compiled keras model 
    
    Parameters
    ----------
    n_layers: int 
        number of hidden layers in model 
        To be clear, this excludes the input and output layer.
        
    first_layer_nodes: int
        Number of nodes in the first hidden layer 

    last_layer_nodes: int
        Number of nodes in the last hidden layer (this is the layer just prior to the output layer)
        
     act_funct: string 
         Name of activation function to use in hidden layers (this excludes the output layer)
        
    Returns
    -------
    model: keras object 
    """
    
    def gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation=True):
        """
        Generates and returns the number of nodes in each hidden layer. 
        To be clear, this excludes the input and output layer. 

        Note
        ----
        Number of nodes in each layer is linearly incremented. 
        For example, gen_layer_nodes(5, 500, 100) will generate [500, 400, 300, 200, 100]

        Parameters
        ----------
        n_layers: int
            Number of hidden layers
            This values should be 2 or greater 

        first_layer_nodes: int

        last_layer_nodes: int

        Returns
        -------
        layers: list of ints
            Contains number of nodes for each layer 
        """

        # throws an error if n_layers is less than 2 
        assert n_layers >= 2, "n_layers must be 2 or greater"

        layers = []

        # PROTIP: IF YOU WANT THE NODE INCREMENTATION TO BE SPACED DIFFERENTLY
        # THEN YOU'LL NEED TO CHANGE THE WAY THAT IT'S CALCULATED - HAVE FUN!
        # when set to True number of nodes are decreased for subsequent layers 
        # NOTE: the order of the number of nodes doesn't matter
        if negative_node_incrementation:
            # subtract this amount from previous layer's nodes in order to increment towards smaller numbers 
            nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
            
        # when set to False number of nodes are increased for subsequent layers
        else:
            # add this amount from previous layer's nodes in order to increment towards larger numbers 
            nodes_increment = (last_layer_nodes + first_layer_nodes)/ (n_layers-1)

        nodes = first_layer_nodes

        for i in range(1, n_layers+1):

            layers.append(math.ceil(nodes))

            # increment nodes for next layer 
            nodes = nodes + nodes_increment

        return layers
    
    # create model
    model = Sequential()
    
    n_nodes = gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation)
    
    for i in range(1, n_layers+1):
        if i==1:
            model.add(Dense(first_layer_nodes, input_dim=X_train.shape[1], activation=act_funct))
        else:
            model.add(Dense(n_nodes[i-1], activation=act_funct))
            
            
    # output layer 
    model.add(Dense(10, # 10 unit/neurons in output layer because we have 10 possible labels to predict  
                    activation='softmax')) # use softmax for a label set greater than 2            
    
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', 
                  optimizer='adam', # adam is a good default optimizer 
                  metrics=['accuracy'])
    
    # do not include model.fit() inside the create_model function
    # KerasClassifier is expecting a complied model 
    return model


## 1.1 Explore `create_model`



The helper function `gen_layer_nodes()` which is contained inside `create_model()` <br>
returns a list containing the number of nodes for each successive layer.<br>

Let's check that `gen_layer_nodes()` behaves as expected. <br>
In other words, we'll perform a **Unit Test!**

In [7]:
    def gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation=True):
        """
        Generates and returns the number of nodes in each hidden layer. 
        To be clear, this excludes the input and output layer. 

        Note
        ----
        Number of nodes in each layer is linearly incremented. 
        For example, gen_layer_nodes(5, 500, 100) will generate [500, 400, 300, 200, 100]

        Parameters
        ----------
        n_layers: int
            Number of hidden layers
            This values should be 2 or greater 

        first_layer_nodes: int

        last_layer_nodes: int

        Returns
        -------
        layers: list of ints
            Contains number of nodes for each layer 
        """

        # throws an error if n_layers is less than 2 
        assert n_layers >= 2, "n_layers must be 2 or greater"

        layers = []

        # PROTIP: IF YOU WANT THE NODE INCREMENTATION TO BE SPACED DIFFERENTLY
        # THEN YOU'LL NEED TO CHANGE THE WAY THAT IT'S CALCULATED - HAVE FUN!
        # when set to True number of nodes are decreased for subsequent layers 
        # NOTE: the order of the number of nodes doesn't matter
        if negative_node_incrementation:
            # subtract this amount from previous layer's nodes in order to increment towards smaller numbers 
            nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
            #print(f'nodes increment = {nodes_increment}')
            
        # when set to False number of nodes are increased for subsequent layers
        else:
            # add this amount from previous layer's nodes in order to increment towards larger numbers 
            nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
            #print(f'nodes increment = {nodes_increment}')

        nodes = first_layer_nodes

        for i in range(1, n_layers+1):

            layers.append(math.ceil(nodes))

            # increment nodes for next layer 
            nodes = nodes + nodes_increment

        return layers




### `negative_node_incrementation = True`
For this case we want the number of nodes to _decrease_ by a constant number for successive layers. <br>So `first_layer_nodes` must be _larger_ than `last_layer_nodes` 

In [8]:
n_layers = 5
first_layer_nodes = 500
last_layer_nodes = 100
negative_node_incrementation = True
n_nodes = gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation)
print(f'Number of nodes in successive layers: {n_nodes}')

Number of nodes in successive layers: [500, 400, 300, 200, 100]


### `negative_node_incrementation = False`
For this case we want the number of nodes to _increase_ by a constant number for successive layers. <br>So `first_layer_nodes` must be _smaller_ than `last_layer_nodes` 

In [9]:
n_layers = 5
first_layer_nodes = 100
last_layer_nodes = 500
negative_node_incrementation = False
n_nodes = gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation)
print(f'Number of nodes in successive layers: {n_nodes}')

Number of nodes in successive layers: [100, 200, 300, 400, 500]


### OK, the Unit Test is passed!

### Let's build a few models<br> 
in order to understand how `create_model()` works in practice. 

### Build a model, setting `negative_node_incrementation = True` 

Use `create_model` to build a model. 

- Set `n_layers = 10` 
- Set `first_layer_nodes = 500`
- Set `last_layer_nodes = 100`
- Set `act_funct = "relu"`


In [10]:
# use create_model to create a model 

# YOUR CODE HERE
model = create_model(n_layers=10,  first_layer_nodes=500, last_layer_nodes=100, act_funct ="relu", negative_node_incrementation=True)

In [11]:
# run model.summary() and make sure that you understand the model architecture that you just built 
# Notice in the model summary how the number of nodes have been linearly incremented in decreasing values. 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 500)               392500    
                                                                 
 dense_1 (Dense)             (None, 456)               228456    
                                                                 
 dense_2 (Dense)             (None, 412)               188284    
                                                                 
 dense_3 (Dense)             (None, 367)               151571    
                                                                 
 dense_4 (Dense)             (None, 323)               118864    
                                                                 
 dense_5 (Dense)             (None, 278)               90072     
                                                                 
 dense_6 (Dense)             (None, 234)               6

### Build a model, setting `negative_node_incrementation = False` 

Use `create_model` to build a model. 

- Set `n_layers = 10` 
- Set `first_layer_nodes = 100`
- Set `last_layer_nodes = 500`
- Set `act_funct = "relu"`

In [12]:
# use create_model to create a model 

# YOUR CODE HERE
model = create_model(n_layers=10,  first_layer_nodes=100, last_layer_nodes=500, act_funct ="relu", negative_node_incrementation=False)

In [13]:
# run model.summary() and make sure that you understand the model architecture that you just built 
# Notice in the model summary how the number of nodes have been linearly incremented in decreasing values. 
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 100)               78500     
                                                                 
 dense_12 (Dense)            (None, 167)               16867     
                                                                 
 dense_13 (Dense)            (None, 234)               39312     
                                                                 
 dense_14 (Dense)            (None, 301)               70735     
                                                                 
 dense_15 (Dense)            (None, 367)               110834    
                                                                 
 dense_16 (Dense)            (None, 434)               159712    
                                                                 
 dense_17 (Dense)            (None, 501)              

## 1.2 Create a grid search using `sklearn`

### Hyperparameter search

In [14]:
# define the grid search parameters
param_grid = {'n_layers': [2, 3],
              'epochs': [3], 
              "first_layer_nodes": [500, 300],
              "last_layer_nodes": [100, 50]
             }

In [15]:
model = KerasClassifier(create_model)

/var/folders/l5/9b73nm7x0d5bz481kpj_k3640000gn/T/ipykernel_7347/1702691071.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(create_model)


In [16]:
%%time
# Create Grid Search
grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    n_jobs=-2, 
                    verbose=1, 
                    cv=3)

grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Fitting 3 folds for each of 8 candidates, totalling 24 fits


2023-08-25 17:49:47.975718: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-08-25 17:49:47.992847: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-08-25 17:49:48.040106: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-08-25 17:49:48.041252: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-08-25 17:49:48.041720: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-08-25 17:49:48.049463: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-08-25 17:49:48.050665: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/3
Epoch 1/3
Epoch 1/3
Epoch 1/3
Epoch 1/3
Epoch 1/3
Epoch 1/3
1563/1563 [==============================] - 6s 4ms/step - loss: 0.6644 - accuracy: 0.7994
Epoch 2/3
1563/1563 [==============================] - 6s 4ms/step - loss: 0.6497 - accuracy: 0.8019
Epoch 2/3
1563/1563 [==============================] - 6s 4ms/step - loss: 0.6460 - accuracy: 0.8048
Epoch 2/3
1563/1563 [==============================] - 8s 5ms/step - loss: 0.6441 - accuracy: 0.8016
Epoch 2/3
1563/1563 [==============================] - 8s 5ms/step - loss: 0.6459 - accuracy: 0.8024
Epoch 2/3
1563/1563 [==============================] - 8s 5ms/step - loss: 0.6500 - accuracy: 0.7993
Epoch 2/3
1563/1563 [==============================] - 6s 4ms/step - loss: 0.4464 - accuracy: 0.8652
Epoch 3/3
1563/1563 [==============================] - 6s 4ms/step - loss: 0.4419 - accuracy: 0.8659
Epoch 3/3
1563/1563 [==============================] - 6s 4ms/step - loss: 0.4421 - accuracy: 0.8667
Epoch 3/3
1563/1563 [==========

2023-08-25 17:50:52.988146: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


2344/2344 [==============================] - 3s 1ms/step - loss: 0.6147 - accuracy: 0.8138
Epoch 2/3
2344/2344 [==============================] - 3s 1ms/step - loss: 0.4238 - accuracy: 0.8722
Epoch 3/3
2344/2344 [==============================] - 3s 1ms/step - loss: 0.3491 - accuracy: 0.8934
Best: 0.8650533358256022 using {'epochs': 3, 'first_layer_nodes': 500, 'last_layer_nodes': 50, 'n_layers': 2}
Means: 0.8640399972597758, Stdev: 0.004030427972440491 with: {'epochs': 3, 'first_layer_nodes': 500, 'last_layer_nodes': 100, 'n_layers': 2}
Means: 0.8609333435694376, Stdev: 0.0031472965886562843 with: {'epochs': 3, 'first_layer_nodes': 500, 'last_layer_nodes': 100, 'n_layers': 3}
Means: 0.8650533358256022, Stdev: 0.0019936730781571197 with: {'epochs': 3, 'first_layer_nodes': 500, 'last_layer_nodes': 50, 'n_layers': 2}
Means: 0.8614933292071024, Stdev: 0.007058330871421605 with: {'epochs': 3, 'first_layer_nodes': 500, 'last_layer_nodes': 50, 'n_layers': 3}
Means: 0.8611466685930887, Stdev:

In [17]:
best_model = grid_result.best_estimator_

In [18]:
best_model.get_params()

{'epochs': 3,
 'first_layer_nodes': 500,
 'last_layer_nodes': 50,
 'n_layers': 2,
 'build_fn': <function __main__.create_model(n_layers, first_layer_nodes, last_layer_nodes, act_funct='relu', negative_node_incrementation=True)>}

Ok, now that we've played around a bit with  `create_model`, let's build a  simpler model that we'll use to run gridsearches. 

### Build model

Use `create_model` to build a model. 

- Set `n_layers = 2` 
- Set `first_layer_nodes = 500`
- Set `last_layer_nodes = 100`
- Set `act_funct = "relu"`
- Make sure that `negative_node_incrementation = True`

In [19]:
# use create_model to create a model 

###BEGIN SOLUTION
model = create_model(n_layers=2,  first_layer_nodes=500, last_layer_nodes=100, act_funct ="relu", negative_node_incrementation=True) 

###END SOLUTION

In [20]:
# run model.summary() and make sure that you understand the model architecture that you just built 
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 500)               392500    
                                                                 
 dense_26 (Dense)            (None, 100)               50100     
                                                                 
 dense_27 (Dense)            (None, 10)                1010      
                                                                 
Total params: 443,610
Trainable params: 443,610
Non-trainable params: 0
_________________________________________________________________


In [21]:
# define the grid search parameters
param_grid = {'n_layers': [2, 3],
              'epochs': [3], 
              "first_layer_nodes": [500, 300],
              "last_layer_nodes": [100, 50]
             }

In [22]:
model = KerasClassifier(create_model)

/var/folders/l5/9b73nm7x0d5bz481kpj_k3640000gn/T/ipykernel_7347/1702691071.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(create_model)


In [23]:
%%time
# Create Grid Search
grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    n_jobs=-2, 
                    verbose=1, 
                    cv=3)

grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Epoch 1/3
Epoch 1/3
Epoch 1/3
Epoch 1/3
1563/1563 [==============================] - 5s 3ms/step - loss: 0.6645 - accuracy: 0.7989
Epoch 2/3
1563/1563 [==============================] - 5s 3ms/step - loss: 0.6478 - accuracy: 0.8035
Epoch 2/3
1563/1563 [==============================] - 5s 3ms/step - loss: 0.6485 - accuracy: 0.8023
Epoch 2/3
1563/1563 [==============================] - 6s 4ms/step - loss: 0.6469 - accuracy: 0.8053
Epoch 2/3
1563/1563 [==============================] - 8s 5ms/step - loss: 0.6531 - accuracy: 0.7999
Epoch 2/3
1563/1563 [==============================] - 8s 5ms/step - loss: 0.6513 - accuracy: 0.7995
Epoch 2/3
1563/1563 [==============================] - 8s 5ms/step - loss: 0.6456 - accuracy: 0.8019
Epoch 2/3
1563/1563 [==============================] - 5s 3ms/step - loss: 0.4505 - accuracy: 0.8655
Epoch 3/3
1563/1563 [==============================] - 5s 4ms/step - loss: 0.4403 - accuracy: 0.8668
E

In [24]:
best_model = grid_result.best_estimator_

In [25]:
best_model.get_params()

{'epochs': 3,
 'first_layer_nodes': 500,
 'last_layer_nodes': 50,
 'n_layers': 3,
 'build_fn': <function __main__.create_model(n_layers, first_layer_nodes, last_layer_nodes, act_funct='relu', negative_node_incrementation=True)>}

-----

# Experiment 2: Run the Gridsearch Algorithms 

In this section, we are going to use the same model and dataset in order to benchmark 3 different gridsearch approaches: 

- Gridsearch
- Random Search
- Bayesian Optimization. 


Our goal in this experiment is two-fold. We want to see which appraoch 

- Scores the highest accuracy
- Has the shortest run time 

We want to see how these 3 gridsearch approaches handle these trade-offs and to give you a sense of those trades offs.

### Trade-offs

`Gridsearch` will train a model on every single unique hyperparameter combination, this guarantees that you'll get the highest possible accuracy from your parameter set but your gridsearch might have a very long run-time. 

`Random Search` will randomly sample from your parameter set which, depending on how many samples, the run-time might be significantly cut down but you might or might not sample the parameters that correspond to the heightest possible accuracies. 

`Bayesian Optimization` has a bit of intelligence built into it's search algorithm but you do need to manually select some parameters which may greatly influence the model learning outcomes. 

-------
### Build our hyperparameter dictionary

In [26]:
# build out our hyperparameter dictionary 
hp = HyperParameters()
hp.Int('units', min_value=32, max_value=512, step=32)
hp.Choice('learning_rate',values=[1e-1, 1e-2, 1e-3])
hp.Choice('activation',values=["relu", "sigmoid"])

'relu'

---------
## 2.1 Gridsearch Optimization


### Populate a `sklearn` compatible parameter dictionary

In [27]:
# build out our hyperparameter dictionary 
hyper_parameters = {
    # BUG Fix: cast array as list otherwise GridSearchCV will throw error
    "units": np.arange(32, 512, 32).tolist(),
    "learning_rate": [1e-1, 1e-2, 1e-3],
    "activation":["relu", "sigmoid"]
}

In [28]:
hyper_parameters

{'units': [32,
  64,
  96,
  128,
  160,
  192,
  224,
  256,
  288,
  320,
  352,
  384,
  416,
  448,
  480],
 'learning_rate': [0.1, 0.01, 0.001],
 'activation': ['relu', 'sigmoid']}

### Build a `sklearn` compatible model function

In [29]:
def build_model(units, learning_rate, activation):
    
    """
    Returns a complie keras model ready for keras-tuner gridsearch algorithms 
    """
    
    model = Sequential()
    
    # hidden layer
    model.add(Dense(units, activation=activation))
    
    # output layer
    model.add(Dense(10, activation='softmax'))
    
    model.compile(
        optimizer=Adam(learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    
    return model

### Apply the "wrapper" to make the model compatible with `sklearn`

In [30]:
model = KerasClassifier(build_fn = build_model)

/var/folders/l5/9b73nm7x0d5bz481kpj_k3640000gn/T/ipykernel_7347/3788592660.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = build_model)


In [31]:
# save start time 
start = time()

# Create Grid Search
grid = GridSearchCV(estimator=model, 
                    param_grid=hyper_parameters, 
                    n_jobs=-2, 
                    verbose=1, 
                    cv=3)

grid_result = grid.fit(X_train, y_train)

# save end time 
end = time()

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Fitting 3 folds for each of 90 candidates, totalling 270 fits


782/782 [==============================] - 1s 850us/step - loss: 2.1239 - accuracy: 0.1747


  50/1563 [..............................] - ETA: 3s - loss: 4.0835 - accuracy: 0.254450

472/782 [=================>............] - ETA: 0s - loss: 1.9974 - accuracy: 0.3004

586/782 [=====================>........] - ETA: 0s - loss: 1.8198 - accuracy: 0.3435

...

 476/1563 [========>.....................] - ETA: 2s - loss: 0.9726 - accuracy: 0.7083

 819/1563 [==============>...............] - ETA: 2s - loss: 0.8999 - accuracy: 0.7320

 868/1563 [===============>..............] - ETA: 1s - loss: 0.8974 - accuracy: 0.7334

782/782 [==============================] - 0s 521us/step - loss: 0.6677 - accuracy: 0.8054


2344/2344 [==============================] - 2s 896us/step - loss: 0.6244 - accuracy: 0.8146
Best: 0.8422533472379049 using {'activation': 'relu', 'learning_rate': 0.001, 'units': 384}
Means: 0.2595333407322566, Stdev: 0.014511287003110076 with: {'activation': 'relu', 'learning_rate': 0.1, 'units': 32}
Means: 0.2139999916156133, Stdev: 0.07359853941055831 with: {'activation': 'relu', 'learning_rate': 0.1, 'units': 64}
Means: 0.23085332910219827, Stdev: 0.024630412461728906 with: {'activation': 'relu', 'learning_rate': 0.1, 'units': 96}
Means: 0.28726667165756226, Stdev: 0.03420459976409997 with: {'activation': 'relu', 'learning_rate': 0.1, 'units': 128}
Means: 0.2630266696214676, Stdev: 0.08953784752345452 with: {'activation': 'relu', 'learning_rate': 0.1, 'units': 160}
Means: 0.24593332906564078, Stdev: 0.06594774122002116 with: {'activation': 'relu', 'learning_rate': 0.1, 'units': 192}
Means: 0.3206266661485036, Stdev: 0.012421760595619738 with: {'activation': 'relu', 'learning_rate'

In [32]:
# total run time 
total_run_time_in_miniutes = (end - start)/60
total_run_time_in_miniutes

3.0616034626960755

In [33]:
grid_result.best_params_

{'activation': 'relu', 'learning_rate': 0.001, 'units': 384}

In [34]:
# because all other optimization approaches are reporting test set score
# let's calculate the test set score in this case 
best_model = grid_result.best_estimator_
test_acc = best_model.score(X_test, y_test)

782/782 [==============================] - 0s 466us/step - loss: 0.4968 - accuracy: 0.8541


In [35]:
test_acc

0.8541200160980225

 ### Results 

Best parameters: 'activation': 'relu', 'learning_rate': 0.001, 'units': 384

Best score: 0.85

------
## 2.2 Random Search with `keras-tuner`

Be sure to check out the [**docs for Keras-Tuner**](https://keras-team.github.io/keras-tuner/documentation/tuners/). Here you can read about the input parameters for the `RandomSearch` tuner.

### Build model
---

In [36]:
# because gridsearching can take a lot of time and we are bench marking 3 different approaches
# let's build a simple model to minimize run time 

def build_model(hp):
    
    """
    Returns a complied keras model ready for keras-tuner gridsearch algorithms 
    """
    
    model = Sequential()
    
    # hidden layer
    model.add(Dense(units=hp.get('units'),activation=hp.get("activation")))
    
    # output layer
    model.add(Dense(10, activation='softmax'))
    
    model.compile(
        optimizer=Adam(hp.get('learning_rate')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    
    return model

In [40]:
# how many unique hyperparameter combinations do we have? 
# HINT: take the product of the number of possible values for each hyperparameter 
# save your answer to n_unique_hparam_combos

# YOUR CODE HERE
n_unique_hparam_combos = 15 * 3 * 2

In [41]:
# how many of these do we want to randomly sample?
# let's pick 25% of n_unique_hparam_combos param combos to sample
# save this number to n_param_combos_to_sample

# YOUR CODE HERE
fraction_to_sample = 0.25
n_param_combos_to_sample = int(fraction_to_sample * n_unique_hparam_combos)
# raise NotImplementedError()

### Instantiate a `RandomSearch()` object for your grid search

In [43]:
random_tuner = RandomSearch(
            build_model,
            objective='val_accuracy',
            max_trials=n_param_combos_to_sample, # number of times to sample the parameter set and build a model 
            seed=1234,
            hyperparameters=hp, # pass in our hyperparameter dictionary
            directory='./keras-tuner-trial',
            project_name='random_search')

In [44]:
 # take note of Total elapsed time in print out -- took ~10 minutes without GPU
random_tuner.search(X_train, y_train,
                    epochs=3,
                    validation_data=(X_test, y_test))

Trial 22 Complete [00h 00m 08s]
val_accuracy: 0.8167200088500977

Best val_accuracy So Far: 0.8727999925613403
Total elapsed time: 00h 02m 38s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [45]:
# identify the best score and hyperparamter (should be at the top since scores are ranked)
random_tuner.results_summary()

Results summary
Results in ./keras-tuner-trial/random_search
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 13 summary
Hyperparameters:
units: 384
learning_rate: 0.001
activation: relu
Score: 0.8727999925613403

Trial 06 summary
Hyperparameters:
units: 416
learning_rate: 0.001
activation: relu
Score: 0.871999979019165

Trial 18 summary
Hyperparameters:
units: 256
learning_rate: 0.001
activation: relu
Score: 0.8681600093841553

Trial 00 summary
Hyperparameters:
units: 512
learning_rate: 0.001
activation: sigmoid
Score: 0.860319972038269

Trial 19 summary
Hyperparameters:
units: 192
learning_rate: 0.001
activation: sigmoid
Score: 0.8545200228691101

Trial 04 summary
Hyperparameters:
units: 96
learning_rate: 0.001
activation: sigmoid
Score: 0.8407999873161316

Trial 16 summary
Hyperparameters:
units: 448
learning_rate: 0.01
activation: sigmoid
Score: 0.8319200277328491

Trial 07 summary
Hyperparameters:
units: 224
learning_rate: 0.01
activation: sigmoid
Scor

 ### Results
 
Identify and write the the best performing hyperparameter combination and model score. 
Note that because this is Random Search, multiple runs might have slighly different outcomes. 
 
 

Best Hyperparameters:

units: 384

learning_rate: 0.001

activation: relu

Score: 0.873

------
## 2.3 Bayesian Optimization with `keras-tuner`

![](https://upload.wikimedia.org/wikipedia/commons/0/02/GpParBayesAnimationSmall.gif)

Be sure to check out the [**docs for Keras-Tuner**](https://keras-team.github.io/keras-tuner/documentation/tuners/). Here you can read about the input parameters for the `BayesianOptimization` tuner.

Pay special attention to these `BayesianOptimization` parameters: `num_initial_points` and `beta`. 

`num_initial_points`: 

Number of randomly selected hyperparameter combinations to try before applying bayesian probability to determine liklihood of which param combo to try next based on expected improvement


`beta`: 

Larger values means more willing to explore new hyperparameter combinations (analogous to searching for the global minimum in Gradient Descent), smaller values means that it is less willing to try new hyperparameter combinations (analogous to getting stuck in a local minimum in Gradient Descent). 

As a start, error on the side of larger values. What defines a small or large value you ask? That question would pull us into the mathematical intricacies of Bayesian Optimization and Gaussian Processes. For simplicity, notice that the default value is 2.6 and work from there. 

In [46]:
# we know that 24 samples is about 25% of 96 possible hyper-parameter combos
# let's set up a run with the same parameters we used for RandomSearch() so the comparison will be aplles-to-apples
# feel free to play with any of these numbers later
max_trials=24
num_initial_points=5
beta=5.0

#### Instantiate a `BayesianOptimization()` object for your grid search

In [47]:
bayesian_tuner = BayesianOptimization(
                    build_model,
                    objective='val_accuracy',
                    max_trials=max_trials,
                    hyperparameters=hp, # pass in our hyperparameter dictionary
                    num_initial_points=num_initial_points, 
                    beta=beta, 
                    seed=1234,
                    directory='./keras-tuner-trial',
                    project_name='bayesian_optimization_4')

In [48]:
bayesian_tuner.search(X_train, y_train,
               epochs=3,
               validation_data=(X_test, y_test))

Trial 24 Complete [00h 00m 09s]
val_accuracy: 0.8737999796867371

Best val_accuracy So Far: 0.8745599985122681
Total elapsed time: 00h 03m 01s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [49]:
bayesian_tuner.results_summary()

Results summary
Results in ./keras-tuner-trial/bayesian_optimization_4
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 22 summary
Hyperparameters:
units: 512
learning_rate: 0.001
activation: relu
Score: 0.8745599985122681

Trial 21 summary
Hyperparameters:
units: 512
learning_rate: 0.001
activation: relu
Score: 0.873960018157959

Trial 23 summary
Hyperparameters:
units: 512
learning_rate: 0.001
activation: relu
Score: 0.8737999796867371

Trial 17 summary
Hyperparameters:
units: 320
learning_rate: 0.001
activation: relu
Score: 0.8720800280570984

Trial 20 summary
Hyperparameters:
units: 512
learning_rate: 0.001
activation: relu
Score: 0.8720399737358093

Trial 09 summary
Hyperparameters:
units: 448
learning_rate: 0.001
activation: relu
Score: 0.8719199895858765

Trial 06 summary
Hyperparameters:
units: 512
learning_rate: 0.001
activation: relu
Score: 0.8678799867630005

Trial 19 summary
Hyperparameters:
units: 512
learning_rate: 0.001
activation: relu
Score

 ### Results
 
Identify and write the the best performing hyperparameter combination and model score. 
Note that because this is  Bayesian Optimization, multiple runs might have slighly different outcomes. 
 
 

Best Hyperparameters:
units: 512

learning_rate: 0.001

activation: relu

Score: 0.875

We should point out that Gridsearch split the training set internally and created a test set whereas keras-tuner allows us to pass in a test set. This means that the keras-tuner algorithms were using one test set and our skearn GridSearchCV was using a different test set - so this isn't a perfectly exact 1-to-1 comparision but it'll have to do. In order to compensate for this, we did score the best model on the same test set that keras-tuner used. 

_______

# Conclusion

The spirit of this experiment is to expose you to the idea of benchmarking and comparing the trade-offs of various gridsearch approaches. 

Even if we did find a way to pass in the original test set into GridSearchCV, we can see that both Random Search and Bayesian Optimization are arguably better alternatives to a brute force grid search when we consider the trade-offs of run time and locating the best performing model. 

----

# Stretch Goals

- Feel free to run whatever gridsearch experiments on whatever models you like!

In [ ]:
# this is your open playground - be free to explore as you wish 